## Memory layouts

We can imagine memory as a linear collection of consecutive memory addresses (each one representing one byte).


![Byte array in memory](../hpc_lecture_notes/img/byte_array.png)

The key toe fficient data representations is to order data spatially local in memory. This means that the data we want to work on next should be as close to our current data as possible. The reason is that memory accesses in modern computers are extremely expensive compared to actual computations. In order to alleviate this problem all modern CPUs rely on sophisticated caches that try to read data ahead of time from the memory. This works only if the next pieces of data are close to the data that we are currently working on.

Standard Python list types do not guarantee this locality. List elements can be at very different memory addresses, making standard lists are other base Python types unsuitable for numerical operations. What we require is a buffer type that guarantees us a chunk of consecutive addresses in the system memory.

What happens if we have a matrix? Consider a 2 x 2 matrix

$$
A = \begin{bmatrix} 1 & 2\\ 3 & 4\end{bmatrix}
$$

We have two ways of ordering this matrix across the memory band

* **C-Style ordering:** This aligns the matrix row by row in memory. Hence, our memory buffer will have four elements that read

$$
1, 2, 3, 4
$$

* **Fortran Style ordering:** This aligns the matrix column by column in memory. Our memory buffer will now have four elements that read

$$
1, 3, 2, 4
$$

Both memory layout styles are used across scientific computing, and it is important to know what the assumed layout in a given numerical library is. Ignoring data layouts leads to inefficiency if code has to translate on the fly between the layouts, or even bugs if the layout differences are ignored by a library.

In [2]:
import numpy as np
import time

In [3]:
N = 10000

Let's create two identical NxN arrays, but store them into memory with C and Fortran style orderings. For the purpose of computation, the arrays are element-wise equal.

In [4]:
A = np.random.rand(N,N)
Ac = np.array(A, order = 'C')
Af = np.array(A, order = 'F')
B = np.zeros(N)

print ('A == Ac is', (A == Ac).all())
print ('A == Af is', (A == Af).all())
print ('Ac == Af is', (Ac == Af).all())

A == Ac is True
A == Af is True
Ac == Af is True


So what is the difference? The strides attribute tells us how many bytes in memory we have to stride to reach the next element *per dimension*.

In [1]:
print('Strides of Ac', Ac.strides)
print('Strides of Af', Af.strides)

NameError: name 'Ac' is not defined

The stride makes a difference when we have to traverse through values of the array because of how the CPU loads data from memory into its registers. Operations are much faster when we can access our data with *unit stride*, that is we are accessing items that are contiguous in memory. For example, reshaping the array is very fast with C ordering (unit stride) and very slow with F ordering (N stride)

In [5]:
%%time
Ac.reshape(N*N,1);

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11 µs


array([[0.87168933],
       [0.86088697],
       [0.53632319],
       ...,
       [0.58328921],
       [0.54419975],
       [0.45465535]])

In [6]:
%%time
Af.reshape(N*N,1);

CPU times: user 1.38 s, sys: 346 ms, total: 1.73 s
Wall time: 2.95 s


array([[0.87168933],
       [0.86088697],
       [0.53632319],
       ...,
       [0.58328921],
       [0.54419975],
       [0.45465535]])

So why care about this? C ordering is the default so just don't touch it and the code will be fast, right? Not quite. It depends on how you are accessing the elements of the array. Of course numpy functions will be optimized for the default ordering, but if you are doing more complex memory accesses it can make a big difference. This is also important to keep in mind if you are interfacing to other languages.

In [8]:
%%time
for i in np.arange(N):
    B[i] = np.sum(Ac[i,:])

CPU times: user 180 ms, sys: 8.02 ms, total: 188 ms
Wall time: 455 ms


In [9]:
%%time
for i in np.arange(N):
    B[i] = np.sum(Af[i,:])

CPU times: user 1.42 s, sys: 46.1 ms, total: 1.47 s
Wall time: 2.85 s


In [10]:
%%time
for i in np.arange(N):
    B[i] = np.sum(Ac[:,i])

CPU times: user 1.5 s, sys: 48.7 ms, total: 1.55 s
Wall time: 2.88 s


In [11]:
%%time
for i in np.arange(N):
    B[i] = np.sum(Af[:,i])

CPU times: user 182 ms, sys: 7.71 ms, total: 190 ms
Wall time: 344 ms
